# ส่วนที่ 3: การโหลดข้อมูลลง SQLite (Data Loading)

Notebook นี้แสดงขั้นตอนการโหลดข้อมูลลงในฐานข้อมูล SQLite และการตรวจสอบความถูกต้องของข้อมูล

## 1. ติดตั้งแพ็คเกจที่จำเป็น

In [ ]:
# ติดตั้งแพ็คเกจที่จำเป็น
!pip install pandas numpy matplotlib seaborn sqlalchemy

## 2. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sqlalchemy import create_engine

# ตั้งค่าการแสดงผลกราฟให้สวยงาม
plt.style.use('ggplot')
sns.set(style='whitegrid')

# ตั้งค่าให้แสดงข้อมูลภาษาไทยได้
plt.rcParams['font.family'] = 'DejaVu Sans'

# แสดงเวอร์ชันของแพ็คเกจ
print(f"Pandas version: {pd.__version__}")
print(f"SQLite version: {sqlite3.sqlite_version}")

## 3. กำหนดพาธของโปรเจค

In [ ]:
# กำหนดพาธหลักสำหรับโปรเจค
project_path = './Employee_Data_Analytics_Project'  # ปรับตามโครงสร้างของคุณ

# แสดงไฟล์ในโปรเจค
print("ไฟล์ในโปรเจค:")
for file in os.listdir(project_path):
    print(f"- {file}")

## 4. โหลดข้อมูลที่แปลงแล้ว

In [ ]:
# โหลดข้อมูลที่แปลงแล้ว
transformed_data_path = f"{project_path}/transformed_employee_data.csv"

if os.path.exists(transformed_data_path):
    df_transformed = pd.read_csv(transformed_data_path)
    print(f"โหลดข้อมูลจาก {transformed_data_path} สำเร็จ")
    print(f"ข้อมูลมีทั้งหมด {df_transformed.shape[0]} แถว และ {df_transformed.shape[1]} คอลัมน์")
else:
    print(f"ไม่พบไฟล์ {transformed_data_path}")
    print("กรุณารันไฟล์ 02_Data_Transformation.ipynb ก่อน")
    # ทางเลือกสำหรับกรณีไม่พบไฟล์
    print("จะโหลดข้อมูลจากไฟล์ต้นฉบับและทำการแปลงข้อมูลอย่างง่ายแทน")
    
    # โหลดข้อมูลต้นฉบับ
    original_data_path = f"{project_path}/dataset_employee.csv"
    df = pd.read_csv(original_data_path)
    
    # แปลงข้อมูลอย่างง่าย
    df_transformed = df.copy()
    df_transformed['YEAR_CE'] = df_transformed['YEAR'] - 543
    
    # แปลงไตรมาสเป็นตัวเลข
    quarter_map = {
        'ไตรมาสที่ 1': 1,
        'ไตรมาสที่ 2': 2,
        'ไตรมาสที่ 3': 3,
        'ไตรมาสที่ 4': 4
    }
    df_transformed['QUARTER_NUM'] = df_transformed['QUARTER'].map(quarter_map)
    
    # สร้างคอลัมน์วันที่โดยประมาณ
    def get_quarter_date(row):
        year = row['YEAR_CE']
        quarter = row['QUARTER_NUM']
        if quarter == 1:
            return f"{year}-02-15"
        elif quarter == 2:
            return f"{year}-05-15"
        elif quarter == 3:
            return f"{year}-08-15"
        elif quarter == 4:
            return f"{year}-11-15"

    df_transformed['ESTIMATED_DATE'] = df_transformed.apply(get_quarter_date, axis=1)
    df_transformed['ESTIMATED_DATE'] = pd.to_datetime(df_transformed['ESTIMATED_DATE'])
    
    # สร้างคอลัมน์ YEAR_QUARTER
    df_transformed['YEAR_QUARTER'] = df_transformed['YEAR'].astype(str) + '-Q' + df_transformed['QUARTER_NUM'].astype(str)
    
    print(f"สร้างข้อมูลทดแทนสำเร็จ: {df_transformed.shape[0]} แถว และ {df_transformed.shape[1]} คอลัมน์")

# โหลดตารางสรุปต่างๆ
region_summary_path = f"{project_path}/region_summary.csv"
time_summary_path = f"{project_path}/time_summary.csv"
province_summary_path = f"{project_path}/province_summary.csv"

# ตรวจสอบและโหลดข้อมูลสรุปตามภูมิภาค
if os.path.exists(region_summary_path):
    df_region = pd.read_csv(region_summary_path)
    print(f"โหลดข้อมูลสรุปตามภูมิภาคจาก {region_summary_path} สำเร็จ")
else:
    print(f"ไม่พบไฟล์ {region_summary_path} จะสร้างข้อมูลสรุปจากข้อมูลที่มี")
    df_region = df_transformed.groupby(['YEAR', 'QUARTER', 'REGION', 'SEX']).agg(
        total_employees=('VALUE', 'sum'),
        avg_employees=('VALUE', 'mean'),
        min_employees=('VALUE', 'min'),
        max_employees=('VALUE', 'max'),
        province_count=('PROVINCE', 'nunique')
    ).reset_index()

# ตรวจสอบและโหลดข้อมูลสรุปตามเวลา
if os.path.exists(time_summary_path):
    df_time = pd.read_csv(time_summary_path)
    print(f"โหลดข้อมูลสรุปตามเวลาจาก {time_summary_path} สำเร็จ")
else:
    print(f"ไม่พบไฟล์ {time_summary_path} จะสร้างข้อมูลสรุปจากข้อมูลที่มี")
    df_time = df_transformed.groupby(['YEAR', 'QUARTER', 'YEAR_QUARTER'])['VALUE'].sum().reset_index()
    df_time.rename(columns={'VALUE': 'total_employees'}, inplace=True)

# ตรวจสอบและโหลดข้อมูลสรุปตามจังหวัด
if os.path.exists(province_summary_path):
    df_province = pd.read_csv(province_summary_path)
    print(f"โหลดข้อมูลสรุปตามจังหวัดจาก {province_summary_path} สำเร็จ")
else:
    print(f"ไม่พบไฟล์ {province_summary_path} จะสร้างข้อมูลสรุปจากข้อมูลที่มี")
    df_province = df_transformed.groupby(['PROVINCE', 'SEX']).agg(
        avg_employees=('VALUE', 'mean'),
        min_employees=('VALUE', 'min'),
        max_employees=('VALUE', 'max'),
        count=('VALUE', 'count')
    ).reset_index()

## 5. แปลง Data Type ให้เหมาะสมก่อนโหลดลงฐานข้อมูล

In [ ]:
# แปลง Data Type ให้เหมาะสมสำหรับ SQLite
# แปลงคอลัมน์วันที่เป็น string ในรูปแบบ ISO
if 'ESTIMATED_DATE' in df_transformed.columns and pd.api.types.is_datetime64_any_dtype(df_transformed['ESTIMATED_DATE']):
    df_transformed['ESTIMATED_DATE'] = df_transformed['ESTIMATED_DATE'].dt.strftime('%Y-%m-%d')
    print("แปลงคอลัมน์ ESTIMATED_DATE เป็น ISO date string")

# ตรวจสอบและจัดการค่า NaN
print("\nตรวจสอบค่า NaN ในแต่ละตาราง:")
print(f"- ตารางหลัก: {df_transformed.isnull().sum().sum()} ค่า")
print(f"- ตารางสรุปตามภูมิภาค: {df_region.isnull().sum().sum()} ค่า")
print(f"- ตารางสรุปตามเวลา: {df_time.isnull().sum().sum()} ค่า")
print(f"- ตารางสรุปตามจังหวัด: {df_province.isnull().sum().sum()} ค่า")

# แทนค่า NaN ด้วย NULL สำหรับ SQLite
# (SQLite จะแปลง NaN เป็น NULL โดยอัตโนมัติเมื่อใช้ pandas to_sql)

## 6. สร้างฐานข้อมูล SQLite และโหลดข้อมูล

In [ ]:
# กำหนดพาธของฐานข้อมูล SQLite
db_path = f"{project_path}/employee_database.db"

# สร้างการเชื่อมต่อกับฐานข้อมูล SQLite
conn = sqlite3.connect(db_path)
print(f"เชื่อมต่อกับฐานข้อมูล {db_path} สำเร็จ")

# สร้างเอนจินสำหรับ SQLAlchemy
engine = create_engine(f'sqlite:///{db_path}')

# โหลดข้อมูลลงในฐานข้อมูล SQLite
try:
    # โหลดข้อมูลหลัก
    df_transformed.to_sql('employee_stats', engine, if_exists='replace', index=False)
    print("โหลดข้อมูลหลักลงฐานข้อมูลสำเร็จ")
    
    # โหลดข้อมูลสรุปตามภูมิภาค
    df_region.to_sql('region_summary', engine, if_exists='replace', index=False)
    print("โหลดข้อมูลสรุปตามภูมิภาคลงฐานข้อมูลสำเร็จ")
    
    # โหลดข้อมูลสรุปตามเวลา
    df_time.to_sql('time_summary', engine, if_exists='replace', index=False)
    print("โหลดข้อมูลสรุปตามเวลาลงฐานข้อมูลสำเร็จ")
    
    # โหลดข้อมูลสรุปตามจังหวัด
    df_province.to_sql('province_summary', engine, if_exists='replace', index=False)
    print("โหลดข้อมูลสรุปตามจังหวัดลงฐานข้อมูลสำเร็จ")
    
    print("\nโหลดข้อมูลทั้งหมดลงฐานข้อมูลสำเร็จ")
except Exception as e:
    print(f"เกิดข้อผิดพลาดในการโหลดข้อมูล: {e}")

## 7. สร้าง Index เพื่อเพิ่มประสิทธิภาพในการสืบค้นข้อมูล

In [ ]:
# สร้าง Index เพื่อเพิ่มประสิทธิภาพในการสืบค้นข้อมูล
try:
    cursor = conn.cursor()
    
    # สร้าง Index สำหรับตารางหลัก
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_year_quarter ON employee_stats (YEAR, QUARTER)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_region ON employee_stats (REGION)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_province ON employee_stats (PROVINCE)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_sex ON employee_stats (SEX)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_estimated_date ON employee_stats (ESTIMATED_DATE)")
    
    # สร้าง Index สำหรับตารางสรุปตามภูมิภาค
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_region_summary_region ON region_summary (REGION)")
    
    # สร้าง Index สำหรับตารางสรุปตามเวลา
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_time_summary_year_quarter ON time_summary (YEAR, QUARTER)")
    
    # สร้าง Index สำหรับตารางสรุปตามจังหวัด
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_province_summary_province ON province_summary (PROVINCE)")
    
    conn.commit()
    print("สร้าง Index ทั้งหมดสำเร็จ")
except Exception as e:
    print(f"เกิดข้อผิดพลาดในการสร้าง Index: {e}")

## 8. ตรวจสอบข้อมูลในฐานข้อมูล

In [ ]:
# ตรวจสอบข้อมูลในฐานข้อมูล
try:
    # ตรวจสอบตารางทั้งหมดในฐานข้อมูล
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    print("ตารางทั้งหมดในฐานข้อมูล:")
    for table in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table[0]}")
        count = cursor.fetchone()[0]
        print(f"- {table[0]}: {count} แถว")
        
        # แสดงโครงสร้างของตาราง
        cursor.execute(f"PRAGMA table_info({table[0]})")
        columns = cursor.fetchall()
        print(f"  ประกอบด้วยคอลัมน์: {len(columns)} คอลัมน์")
        
        # แสดงตัวอย่างข้อมูล 3 แถวแรก
        query = f"SELECT * FROM {table[0]} LIMIT 3"
        sample_data = pd.read_sql_query(query, conn)
        print(f"  ตัวอย่างข้อมูล {table[0]}:")
        display(sample_data)
except Exception as e:
    print(f"เกิดข้อผิดพลาดในการตรวจสอบข้อมูล: {e}")

## 9. ทดสอบการสืบค้นข้อมูลด้วย SQL

In [ ]:
# ทดสอบการสืบค้นข้อมูลด้วย SQL
try:
    # 1. จำนวนพนักงานทั้งหมดแยกตามภูมิภาค
    query1 = """
    SELECT REGION, SUM(VALUE) as TOTAL_EMPLOYEES
    FROM employee_stats
    GROUP BY REGION
    ORDER BY TOTAL_EMPLOYEES DESC
    """
    result1 = pd.read_sql_query(query1, conn)
    print("1. จำนวนพนักงานทั้งหมดแยกตามภูมิภาค:")
    display(result1)
    
    # 2. จำนวนพนักงานแยกตามเพศในแต่ละปีและไตรมาส
    query2 = """
    SELECT YEAR, QUARTER, SEX, SUM(VALUE) as TOTAL_EMPLOYEES
    FROM employee_stats
    GROUP BY YEAR, QUARTER, SEX
    ORDER BY YEAR, QUARTER, SEX
    """
    result2 = pd.read_sql_query(query2, conn)
    print("\n2. จำนวนพนักงานแยกตามเพศในแต่ละปีและไตรมาส:")
    display(result2.head(10))
    
    # 3. จังหวัดที่มีพนักงานมากที่สุด 10 อันดับแรก
    query3 = """
    SELECT PROVINCE, SUM(VALUE) as TOTAL_EMPLOYEES
    FROM employee_stats
    GROUP BY PROVINCE
    ORDER BY TOTAL_EMPLOYEES DESC
    LIMIT 10
    """
    result3 = pd.read_sql_query(query3, conn)
    print("\n3. จังหวัดที่มีพนักงานมากที่สุด 10 อันดับแรก:")
    display(result3)
    
    # 4. แนวโน้มจำนวนพนักงานตามไตรมาส
    query4 = """
    SELECT YEAR, QUARTER, SUM(VALUE) as TOTAL_EMPLOYEES
    FROM employee_stats
    GROUP BY YEAR, QUARTER
    ORDER BY YEAR, QUARTER
    """
    result4 = pd.read_sql_query(query4, conn)
    print("\n4. แนวโน้มจำนวนพนักงานตามไตรมาส:")
    display(result4.head(10))
    
    # 5. สัดส่วนชาย-หญิงในแต่ละภูมิภาค
    query5 = """
    SELECT REGION, SEX, SUM(VALUE) as TOTAL_EMPLOYEES,
           ROUND(SUM(VALUE) * 100.0 / (
               SELECT SUM(VALUE) FROM employee_stats e2 WHERE e2.REGION = e1.REGION
           ), 2) as PERCENTAGE
    FROM employee_stats e1
    GROUP BY REGION, SEX
    ORDER BY REGION, SEX
    """
    result5 = pd.read_sql_query(query5, conn)
    print("\n5. สัดส่วนชาย-หญิงในแต่ละภูมิภาค:")
    display(result5)
except Exception as e:
    print(f"เกิดข้อผิดพลาดในการสืบค้นข้อมูล: {e}")

## 10. สรุปผลการโหลดข้อมูลลงฐานข้อมูล

In [ ]:
# สรุปผลการโหลดข้อมูลลงฐานข้อมูล
print("สรุปผลการโหลดข้อมูลลงฐานข้อมูล:")
print(f"1. สร้างฐานข้อมูล {db_path} สำเร็จ")
print(f"2. โหลดข้อมูลลงในตาราง:")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table[0]}")
    count = cursor.fetchone()[0]
    print(f"   - {table[0]}: {count} แถว")
print("3. สร้าง Index เพื่อเพิ่มประสิทธิภาพในการสืบค้นข้อมูลสำเร็จ")
print("4. ทดสอบการสืบค้นข้อมูลด้วย SQL สำเร็จ")
print("\nข้อมูลพร้อมสำหรับการวิเคราะห์และการสร้างแดชบอร์ด")

# ปิดการเชื่อมต่อกับฐานข้อมูล
conn.close()
print("ปิดการเชื่อมต่อกับฐานข้อมูลสำเร็จ")

## 11. ข้อมูลเสร็จพร้อมสำหรับการวิเคราะห์และการสร้างแดชบอร์ด

ดำเนินการโหลดข้อมูลลงฐานข้อมูล SQLite เสร็จสิ้น ข้อมูลพร้อมสำหรับขั้นตอนการวิเคราะห์และการสร้างแดชบอร์ดใน Power BI ในไฟล์ `04_Data_Visualization.ipynb` และ `05_PowerBI_Design_Guide.ipynb`